In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

# Actual data from 2006 to 2025
years = list(range(2006, 2026))
remaining_forest_percentages = [
    19.43, 19.00, 19.37, 19.75, 19.46, 18.53, 18.67,
    18.24, 18.26, 18.45, 18.52, 18.28, 18.26,
    18.37, 18.09, 18.37, 18.57, 18.18, 18.38, 18.03
]

# Training data up to 2023
training_years = list(range(2006, 2024))
training_data = remaining_forest_percentages[:len(training_years)]
test_data = remaining_forest_percentages[len(training_years):]  # Actual values for 2024–2025

# Normalize the data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(np.array(training_data).reshape(-1, 1))

# Prepare sequences (lookback = 3)
lookback = 10
X, y = [], []
for i in range(lookback, len(scaled_data)):
    X.append(scaled_data[i - lookback:i, 0])
    y.append(scaled_data[i, 0])
X, y = np.array(X), np.array(y)
X = X.reshape((X.shape[0], X.shape[1], 1))

# Build LSTM model
model = Sequential([
    LSTM(50, return_sequences=False, input_shape=(lookback, 1)),
    Dense(1)
])
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X, y, epochs=200, batch_size=1, verbose=0)

# Forecast for 2024 and 2025
forecast_scaled = []
input_seq = scaled_data[-lookback:].reshape(1, lookback, 1)

for _ in range(2):  # Forecast 2 steps ahead
    pred = model.predict(input_seq, verbose=0)[0][0]
    forecast_scaled.append(pred)
    input_seq = np.append(input_seq[:, 1:, :], [[[pred]]], axis=1)

# Inverse transform forecasted values
forecast = scaler.inverse_transform(np.array(forecast_scaled).reshape(-1, 1)).flatten()
forecast_years = [2024, 2025]

# Plotting
plt.figure(figsize=(10, 5))

# Actual data
plt.plot(years, remaining_forest_percentages, marker='o', color='blue', label='Actual (2006–2025)')
for x, y_val in zip(years, remaining_forest_percentages):
    plt.text(x, y_val - 0.2, f'{y_val:.2f}', ha='center', va='bottom', fontsize=8, color='blue')

# Forecasted values
plt.plot(forecast_years, forecast, marker='o', linestyle='--', color='green', label='LSTM Forecast (2024–2025)')
for x, y_val in zip(forecast_years, forecast):
    plt.text(x, y_val + 0.3, f'{y_val:.2f}', ha='center', va='top', fontsize=8, color='green')

plt.axvline(x=2024, linestyle='--', color='gray', label='Forecast Start')

plt.title("LSTM Forecast of Remaining Forest Percentage in Sundarban (2006–2025)")
plt.xlabel("Year")
plt.ylabel("Forest Percentage")
plt.xticks(ticks=range(2006, 2026), rotation=45)
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

# Print forecast vs actual
print("\nLSTM Forecast vs Actual (2024–2025):")
for i, year in enumerate(forecast_years):
    predicted = forecast[i]
    actual = test_data[i]
    diff = abs(predicted - actual)
    comment = "very close!" if diff < 0.05 else "a small gap"
    print(f"{year}: Forecast = {predicted:.2f}%, Actual = {actual:.2f}%, "
          f"Difference = {diff:.2f}% ({comment})")
